In [1]:
using FixedSizeArrays
using LightGraphs
using Quaternions

In [2]:
importall Base

In [3]:
immutable CartesianFrame3D
    name::ASCIIString
end
typealias CF3D CartesianFrame3D

immutable Point3D{T}
    frame::CartesianFrame3D
    v::Vec{3, T}
end

In [4]:
immutable Transform3D{T}
    from::CartesianFrame3D
    to::CartesianFrame3D
    rot::Quaternion{T}
    trans::Vec{3, T}
end

Transform3D(from::CF3D, to::CF3D, rot::Quaternion) = Transform3D(from, to, rot, zero(Vec{3, real(typeof(rot))}))
Transform3D(from::CF3D, to::CF3D, trans::Vec{3}) = Transform3D(from, to, one(Quaternion{eltype(trans)}), trans)

function *(t1::Transform3D, t2::Transform3D)
    @assert t1.from == t2.to
    return Transform3D(t2.from, t1.to, t1.rot * t2.rot, t1.trans + Mat(rotationmatrix(t1.rot)) * t2.trans)
end

function *(t::Transform3D, point::Point3D)
    @assert t1.from == point.frame
    return Point3D(t.to, Mat(rotationmatrix(t.rot)) * point.v + t.trans)
end

function inv{T}(t::Transform3D{T})
    rotinv = inv(t.rot)
    transinv = -Mat(rotationmatrix(rotinv)) * t.trans
    return Transform3D(t.to, t.from, rotinv, transinv)
end

inv (generic function with 38 methods)

In [5]:
f1 = CF3D("1")
f2 = CF3D("2")
f3 = CF3D("3")
f4 = CF3D("4")

CartesianFrame3D("4")

In [6]:
t1 = Transform3D(f2, f1, nquatrand(), rand(Vec{3, Float64}))
v = Point3D(f2, rand(Vec{3, Float64}))
w = t1 * v

Point3D{Float64}(CartesianFrame3D("1"),FixedSizeArrays.Vec{3,Float64}((0.2329775714340946,0.42269110678099353,0.15994330521858613)))

In [7]:
t2 = Transform3D(f3, f2, nquatrand())
t3 = t1 * t2

Transform3D{Float64}(CartesianFrame3D("3"),CartesianFrame3D("1"),-0.918680563408273 - 0.2211261682312591im - 0.25915916837277203jm - 0.19991439564861868km,FixedSizeArrays.Vec{3,Float64}((0.4869441529473808,0.8102239231159727,0.20910141594157627)))

In [8]:
t4 = Transform3D(f4, f3, Vec(1.0, 2.0, 3.0))
t5 = t3 * t4

Transform3D{Float64}(CartesianFrame3D("4"),CartesianFrame3D("1"),-0.918680563408273 - 0.2211261682312591im - 0.25915916837277203jm - 0.19991439564861868km,FixedSizeArrays.Vec{3,Float64}((2.4610280601974237,2.028694432885757,3.1447993472617872)))

In [9]:
inv(t5) * t5

Transform3D{Float64}(CartesianFrame3D("4"),CartesianFrame3D("4"),1.0 + 0.0im + 2.0816681711721685e-17jm + 0.0km,FixedSizeArrays.Vec{3,Float64}((0.0,0.0,0.0)))

In [10]:
immutable SpatialInertia{T}
    frame::CartesianFrame3D
    moment::Mat{3, 3, T}
    centerOfMass::Vec{3, T}
    mass::T
end

immutable RigidBody
    name::ASCIIString
    inertia::Nullable{SpatialInertia{Float64}}
end
RigidBody(name::ASCIIString) = RigidBody(name, Nullable{SpatialInertia{Float64}}())

immutable SpatialMotionMatrix{N, T}
    body::RigidBody
    base::RigidBody
    frame::CartesianFrame3D
    angular::Vec{N, T}
    linear::Vec{N, T}
end

typealias SpatialMotionVector{T} SpatialMotionMatrix{1, T}

abstract Joint

immutable QuaternionFloatingJoint <: Joint
    name::ASCIIString
end

joint_transform{T}(j::QuaternionFloatingJoint, q::Vector{T}) = Transform3D(Quaternion(q[1], q[2 : 4]))
num_positions(joint::QuaternionFloatingJoint) = 7
num_velocities(joint::QuaternionFloatingJoint) = 6

immutable FixedAxisOneDoFJoint <: Joint
    name::ASCIIString
    axis::SpatialMotionVector{Float64}
end

joint_transform{T}(j::FixedAxisOneDoFJoint, q::Vector{T}) = Transform3D(qrotation(j.axis.angular, q[1]), q[1] * j.axis.linear)
num_positions(joint::FixedAxisOneDoFJoint) = 1
num_velocities(joint::FixedAxisOneDoFJoint) = 1

type Mechanism
    graph::DiGraph
    indexToJoint::Dict{Pair{Int64,Int64}, Joint}
    jointToIndex::Dict{Joint, Pair{Int64,Int64}}
    joints::Vector{Joint}
    bodies::Vector{RigidBody}
    bodyToIndex::Dict{RigidBody, Int64}
    
    Mechanism() = new(DiGraph(), Dict(), Dict(), Vector(), Vector(), Dict())
end

function add_body!(m::Mechanism, b::RigidBody)
    index = add_vertex!(m.graph)
    push!(m.bodies, b)
    m.bodyToIndex[b] = index
    return b
end

function add_joint!(m::Mechanism, j::Joint, predecessor::RigidBody, successor::RigidBody)
    index = add_edge!(m.graph, m.bodyToIndex[predecessor], m.bodyToIndex[successor])
    push!(m.joints, j)
    m.indexToJoint[index] = j
    m.jointToIndex[j] = index
    return j
end

add_joint! (generic function with 1 method)

In [17]:
function transform{T}(inertia::SpatialInertia{T}, t::Transform3D{T})
    @assert t.from == inertia.frame
    
    function vector_to_skew_symmetric_squared(a::Vec{3, T})
        aSq = a .* a
        b11 = -aSq[2] - aSq[3]
        b12 = a[1] * a[2]
        b13 = a[1] * a[3]
        b22 = -aSq[1] - aSq[3]
        b23 = a[2] * a[3]
        b33 = -aSq[1] - aSq[2]
        return @fsa([b11 b12 b13; b12 b22 b23; b13 b23 b33])
    end

    J = inertia.moment
    m = inertia.mass
    c = inertia.centerOfMass

    R = Mat(rotationmatrix(t.rot))
    p = t.trans
    
    cnew = R * (c * m)
    Jnew = vector_to_skew_symmetric_squared(cnew)
    cnew += m * p
    Jnew -= vector_to_skew_symmetric_squared(cnew)
    Jnew /= m
    Jnew += R * J * R'
    cnew /= m
    
    return SpatialInertia(t.to, Jnew, cnew, m)
end

transform (generic function with 1 method)

In [18]:
J = rand(Mat{3, 3, Float64}); J = J * J'
c = rand(Vec{3, Float64})
m = rand()
inertia = SpatialInertia(f3, J, c, m)
m = Mechanism()
world = add_body!(m, RigidBody("world"))
body = add_body!(m, RigidBody("body", inertia))
add_joint!(m, QuaternionFloatingJoint("joint"), world, body)

QuaternionFloatingJoint("joint")

In [19]:
transform(inertia, t3)

SpatialInertia{Float64}(CartesianFrame3D("1"),FixedSizeArrays.Mat{3,3,Float64}(
    2.1110507841654718 -0.15084207701833408 0.2291033371724545
    -0.15084207701833408 1.6658917979997854 -0.19849543146361814
    0.2291033371724545 -0.19849543146361837 2.4209042151855495
)
,FixedSizeArrays.Vec{3,Float64}((0.9679131177895178,1.4783241032192054,1.2543099427605198)),0.6484965169518915)

In [ ]:
type MechanismState{T}
    q::Vector{T}
    v::Vector{T}
end